### 1. 引入库

In [ ]:
import torch
import torch.nn as nn
import numpy as np
from torchvision import datasets
from torchvision import transforms
from torch.utils.data.sampler import SubsetRandomSampler

### 2. 选择设备（通常是GPU）
`torch.cuda.is_availabel()`返回一个布尔值（`True`或`False`），是否支持CUDA。  
使用CUDA可以显著提高并行计算速度。

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

### 3. 加载数据集

#### 3.1 定义获取训练集和验证集的数据加载器

In [ ]:
def get_train_val_loader(data_dir, batch_size, augment,
                         random_seed, valid_size = 0.1, shuffle = True):

  # ------------- 设置图像变换 ------------- #
  # (1) 归一化
  normalize = transforms.Normalize(mean = [0.4914, 0.4822, 0.4465],
                                   std = [0.2023, 0.1994, 0.2010])
  # (2) 验证集图像变换
  val_transform = transforms.Compose([transforms.Resize(227),
                                   transforms.ToTensor(),
                                   normalize])
  # (3) 训练集是否数据增强
  if augment:
    train_transform = transforms.Compose([transforms.RandomHorizontalFlip(),
                                      transforms.Resize(227),
                                      transforms.ToTensor(),
                                      normalize])
  else:
    train_transform = transforms.Compose([transforms.Resize(227),
                                       transforms.ToTensor(),
                                       normalize])
  # ---------- 👆 数据变换设置完毕 -------------- #

  # 下载并加载训练集
  train_dataset = datasets.CIFAR10(root = data_dir,
                                  train = True,
                                  download = True,
                                  transform = train_transform)
  val_dataset = datasets.CIFAR10(root = data_dir,
                                  train = True,
                                  download = True,
                                  transform = val_transform)

  # ---------- 划分验证集和训练集 ----------  #
  # (1) 计算训练集图片数量
  num_train = len(train_dataset)
  # (2) 计算验证集数量，并向下取整
  num_val = np.floor(valid_size * num_train)
  # (3) 设置训练集和验证集的划分界限
  split = int(num_val)
  # (4) 生成一个列表索引，其内容为 0 ~ (num_train - 1) 的全部整数
  indices = list(range(num_train))    # 为数据"洗牌"做准备
  if shuffle:
    np.random.seed(random_seed) # 根据种子生成随机数
    np.random.shuffle(indices)  # 根据随机数打乱图片
  # (5) 划分验证集和训练集(根据索引列表 indices 和划分界限 split 划分)
  val_idx = indices[:split]     # 验证集索引列别
  train_idx = indices[split:]   # 训练集索引列表
  # (6) 根据验证集和训练集的索引列表采样数据
  train_sampler = SubsetRandomSampler(train_idx)
  val_sampler = SubsetRandomSampler(val_idx)
  # ---------- 👆 训练集和验证集划分完毕 ---------- #

  # 设置数据加载器
  train_loader = torch.utils.data.DataLoader(train_dataset,
                                             batch_size = batch_size,
                                             sampler = train_sampler)
  val_loader = torch.utils.data.DataLoader(val_dataset,
                                           batch_size = batch_size,
                                           sampler = val_sampler)

  return (train_loader, val_loader)

在论文中，作者提到了本地归一化。`mean = [0.4914, 0.4822, 0.4465], ` `std = [0.2023, 0.1994, 0.2010]` 看不懂不用担心。深度学习以后就不这么用了。

#### 3.2 定义获取测试集的数据加载器

In [ ]:
def get_test_loader(data_dir, batch_size, shuffle = True):

  # ------------- 设置图像变换 ------------- #
  # 归一化
  normalize = transforms.Normalize(mean=[0.485, 0.456, 0.406],
                                  std=[0.229, 0.224, 0.225],)
  # 图像变换
  test_transform = transforms.Compose([transforms.Resize(227),
                                transforms.ToTensor(),
                                normalize])

  # 下载并加载测试集
  test_dataset = datasets.CIFAR10(root = data_dir,
                                  train = False,
                                  download = True,
                                  transform = test_transform)

  # 加载测试数据
  test_loader = torch.utils.data.DataLoader(test_dataset,
                                            batch_size = batch_size,
                                            shuffle = shuffle)

  return test_loader

#### 3.3 调用函数，加载数据

In [ ]:
# 设置数据集下载路径
data_dir = "./data"
# 设置批尺寸
batch_size = 64
# 调用训练集和验证集的 DataLoader
train_loader, val_loader = get_train_val_loader(data_dir = data_dir,
                                                batch_size = batch_size,
                                                augment = True,
                                                random_seed = 1)
# 调用测试集的 DataLoader
test_loader = get_test_loader(data_dir = data_dir,
                              batch_size = batch_size,
                              shuffle = False)

### 4. 构建AlexNet

In [ ]:
class AlexNet(nn.Module):
    def __init__(self, num_classes=10):
        super(AlexNet, self).__init__()
        self.conv_block1 = nn.Sequential(
                        nn.Conv2d(3, 96, kernel_size=11, stride=4, padding=0),
                        nn.BatchNorm2d(96),
                        nn.ReLU()
                        )
        self.conv_block2 = nn.Sequential(
                        nn.Conv2d(96, 256, kernel_size=5, stride=1, padding=2),
                        nn.BatchNorm2d(256),
                        nn.ReLU()
                        )
        self.conv_block3 = nn.Sequential(
                        nn.Conv2d(256, 384, kernel_size=3, stride=1, padding=1),
                        nn.BatchNorm2d(384),
                        nn.ReLU()
                        )
        self.conv_block4 = nn.Sequential(
                        nn.Conv2d(384, 384, kernel_size=3, stride=1, padding=1),
                        nn.BatchNorm2d(384),
                        nn.ReLU()
                        )
        self.conv_block5 = nn.Sequential(
                        nn.Conv2d(384, 256, kernel_size=3, stride=1, padding=1),
                        nn.BatchNorm2d(256),
                        nn.ReLU()
                        )
        self.pool = nn.MaxPool2d(kernel_size = 3, stride = 2)
        self.fc1 = nn.Sequential(nn.Dropout(0.5),nn.Linear(9216, 4096),nn.ReLU())
        self.fc2 = nn.Sequential(nn.Dropout(0.5),nn.Linear(4096, 4096),nn.ReLU())
        self.fc3 = nn.Sequential(nn.Linear(4096, num_classes))

    def forward(self, x):
        out = self.conv_block1(x)
        out = self.pool(out)
        out = self.conv_block2(out)
        out = self.pool(out)
        out = self.conv_block3(out)
        out = self.conv_block4(out)
        out = self.conv_block5(out)
        out = self.pool(out)
        out = out.reshape(out.size(0), -1)
        out = self.fc1(out)
        out = self.fc2(out)
        out = self.fc3(out)
        return out

*   `Dropout`：让一部分神经元输出为0. 目前认为它是一个正则项，防止过拟合。目前由于很少使用全连接了，所以`Dropout`不是那么重要了。



### 5. 设置超参数

In [ ]:
num_classes = 10
epochs = 20
learning_rate = 0.005

model = AlexNet(num_classes).to(device)

# 设置损失函数
cost = nn.CrossEntropyLoss()

# 设置优化器
optimizer = torch.optim.SGD(model.parameters(),     # SGD 随机梯度下降
                            lr = learning_rate,
                            weight_decay = 0.005,   # 正则化项的权重是 0.005
                            momentum = 0.9)

# 一 epoch 训练的总 step 数
train_step = len(train_loader)


*   `weight_decay`：是L2的正则项。
*   **正则化**：在机器学习中，正则化是一种用于防止过拟合的技术。限制模型的复杂性，防止模型对训练数据中的噪声过于敏感，从而提高其在未知数据上的泛化能力。在 AlexNet 提出那年（2012），人们普遍认为正则化对解决模型过拟合问题是很重要的。但在后期，这个观点被推翻了。取而代之的是，网络的设计对防止过拟合更重要的。
*   `momentum`：动量是一种优化算法中常用的技术，通常与随机梯度下降（SGD）结合使用，用于加速模型的训练过程。它的功能是**避免因下降曲线不平滑而落入局部最优解中。**动量的引入主要是为了解决随机梯度下降的一些问题，例如在梯度更新中存在的震荡和收敛速度慢的问题。动量算法引入了一个指数衰减的累积变量，用来持续跟踪梯度的历史信息。这个累积变量就是动量。动量在更新参数时不仅考虑当前梯度，还考虑了之前梯度的方向。这有助于平滑更新过程，减少参数更新的震荡，提高模型训练的稳定性和速度。



### 6. 训练和验证
训练需要 2 个循环的嵌套：外部循环用于循环 epoch ；内部循环用于循环每个 epoch 中的每个 step.

In [ ]:
for epoch in range(epochs):
  for i, (images, labels) in enumerate(train_loader):
    images = images.to(device)
    labels = labels.to(device)

    outputs = model(images)
    loss = cost(outputs, labels)

    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

  print("Epoch [{}/{}], Step [{}/{}], Loss:{:.4f}".format(
      epoch+1, epochs, i+1, train_step, loss.item()))   # loss是张量，需要.item()转为浮点型

  # 一个epoch完成之后，进入验证
  with torch.no_grad():
    correct = 0
    total = 0
    for images, labels in val_loader:
      images = images.to(device)
      labels = labels.to(device)
      outputs = model(images)
      _, predicted = torch.max(outputs.data, 1)
      total += labels.size(0)
      correct += (predicted == labels).sum().item()
      # del images, labels, outputs   # 删除变量以释放内存

    # 输出验证结果
    print("Accuracy on validation: {} %".format(100 * (correct / total)))


*   `torch.max(outputs.data, 1)`返回每一行的最大值以及这些最大值所在的索引。第一个返回值（`_`）是最大值，第二个返回值（`predicted`）是最大值的索引。在这种情况下，我们只关心索引，因为它表示了模型的预测类别。
*   `labels.size(0)`返回的是当前批次中标签的数量。
*   `del images, labels, outputs` 是手动删除变量以释放内存。在Python中，这通常是不必要的，因为Python的垃圾回收器会自动处理不再使用的对象。
*   `for images, labels in val_loader:`不需要`numerate(val_loader)`是因为验证阶段不需要索引。

### 7. 测试

In [ ]:
with torch.no_grad():
  correct = 0
  total = 0
  for images, labels in test_loader:
    images = images.to(device)
    labels = labels.to(device)
    outputs = model(images)
    _, predicted = torch.max(outputs.data, 1)
    total += labels.size(0)
    correct += (predicted == labels).sum().item()
    # del images, labels, outputs   # 删除变量以释放内存

  # 输出测试结果
  print("Accuracy on test: {} %".format(100 * (correct / total)))